In [3]:
import os

In [4]:
%pwd

'd:\\ML\\spelling correction\\Spelling-correction-project\\resreach'

In [5]:
os.chdir("../")

In [6]:
%pwd

'd:\\ML\\spelling correction\\Spelling-correction-project'

In [7]:
from dataclasses import*
from pathlib import Path
@dataclass(frozen=True)

class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metrics_path: Path
    


In [8]:
from SpellingCorrection.constants import*
from SpellingCorrection.utils.common import read_yaml,create_directories

In [9]:
import os
import yaml
from dataclasses import dataclass 
from pathlib import Path
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config["artifacts_root"]])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config["model_trainer"]
        params = self.params["Seq2SeqTrainingArguments"]
        create_directories([config["root_dir"]])

        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config["root_dir"]),
            data_path=Path(config["data_path"]),
            model_ckpt=config["model_ckpt"],
            eval_strategy=params["eval_strategy"],
            eval_steps=params["eval_steps"],
            per_device_train_batch_size=params["per_device_train_batch_size"],
            per_device_eval_batch_size=params["per_device_eval_batch_size"],
            num_train_epochs=params["num_train_epochs"],
            save_steps=params["save_steps"],
            save_total_limit=params["save_total_limit"],
            logging_steps=params["logging_steps"],
            predict_with_generate=params["predict_with_generate"],
            fp16=params["fp16"]
        )
        return model_trainer_config

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config["model_evaluation"]
        create_directories([config["root_dir"]])
        return ModelEvaluationConfig(
            root_dir=Path(config["root_dir"]),
            data_path=Path(config["data_path"]),
            model_path=Path(config["model_path"]),
            tokenizer_path=Path(config["tokenizer_path"]),
            metrics_path=Path(config["root_dir"]) / config["metric_file_name"]
        )

In [10]:
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from datasets import Dataset
import pandas as pd

from transformers import pipeline, set_seed, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import matplotlib.pyplot as plt
import pandas as pd
import torch
from tqdm import tqdm
import os


[2024-06-10 19:10:04,171 : INFO : config : PyTorch version 2.3.0 available.]


In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def evaluate(self):
        # Load tokenizer and model
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        
        # Load dataset
        dataset = load_from_disk(self.config.data_path)
        test_dataset = dataset["test"]

        # Setup training arguments for evaluation
        training_args = Seq2SeqTrainingArguments(
            output_dir=self.config.root_dir,
            per_device_eval_batch_size=16,
            predict_with_generate=True,
            fp16=True
        )

        # Initialize the trainer
        trainer = Seq2SeqTrainer(
            model=model,
            args=training_args,
            tokenizer=tokenizer,
            eval_dataset=test_dataset
        )

        # Evaluate the model
        metrics = trainer.evaluate()
        
        # Save metrics
        metrics_path = os.path.join(self.config.root_dir, self.config.metric_file_name)
        with open(metrics_path, 'w') as f:
            for key, value in metrics.items():
                f.write(f"{key},{value}\n")

        print(f"Evaluation metrics saved to {metrics_path}")

In [12]:
try:
    config=ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation_config=ModelEvaluation(model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2024-06-10 19:10:05,395 : INFO : common : Reading the yaml file from the path: config\config.yaml loaded successfully]
[2024-06-10 19:10:05,395 : INFO : common : Reading the yaml file from the path: params.yaml loaded successfully]


BoxKeyError: "'ConfigBox' object has no attribute 'artifacts_root'"